# 网络与Web编程
---

### 实现 TCP 服务器

举例实现一个简单的应答服务器

In [1]:
from socketserver import BaseRequestHandler


class EchoHandler(BaseRequestHandler):

    def handle(self):
        print('Got connection from', self.client_address)
        while True:
            msg = self.request.recv(8192)
            if not msg:
                break
            self.request.send(msg)

多线程服务端

In [2]:
from socketserver import TCPServer
from threading import Thread

# 线程数
NWORKERS = 16

# 是否允许地址的重用
TCPServer.allow_reuse_address = True
serv = TCPServer(('', 20000), EchoHandler, 
                 # 是否立即绑定并激活 Socket
                 bind_and_activate=False)
# 手动绑定和激活
serv.server_bind()
serv.server_activate()

for _建立多线程服务端 in range(NWORKERS):
    t = Thread(target=serv.serve_forever)
    t.daemon = True
    t.start()

# serv.serve_forever()

客户端

In [3]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 20000))

Got connection from ('127.0.0.1', 49201)


In [4]:
s.send(b'Hello')

5

In [5]:
s.recv(8192)

b'Hello'

---
使用 socket 直接编程实现同样的应答服务器

In [6]:
from socket import socket, AF_INET, SOCK_STREAM

def echo_handler(address, client_sock):
    print('Got connection from {}'.format(address))
    while True:
        msg = client_sock.recv(8192)
        if not msg:
            break
        client_sock.sendall(msg)
    client_sock.close()

def echo_server(address, backlog=5):
    sock = socket(AF_INET, SOCK_STREAM)
    sock.bind(address)
    sock.listen(backlog)
    while True:
        client_sock, client_addr = sock.accept()
        echo_handler(client_addr, client_sock)

---
实现将一个类文件接口放置在底层 Socket 上

In [7]:
import socket
from socketserver import StreamRequestHandler, TCPServer


class EchoHandler(StreamRequestHandler):
    # 以下------是可选配置
    # 所有 Socket 操作的超时时间
    timeout = 5
    # 读缓冲区大小
    rbufsize = -1
    # 写缓冲区大小
    wbufsize = 0
    # 设置 TCP_NODELAY 配置
    disable_nagle_algorithm = False

    def handle(self):
        print('Got connection from', self.client_address)
        try:
            for line in self.rfile:
                # self.wfile 是文件类型对象
                self.wfile.write(line)
        except socket.timeout:
            print('Timed out!')

---

### 实现 UDP 服务器

举例实现一个获取时间的服务器

In [8]:
from socketserver import BaseRequestHandler
import time


class TimeHandler(BaseRequestHandler):

    def handle(self):
        print('Got connection from', self.client_address)
        msg, sock = self.request
        resp = time.ctime()
        sock.sendto(resp.encode('ascii'), self.client_address)

多线程服务端

In [9]:
from socketserver import UDPServer
from threading import Thread

# 线程数
NWORKERS = 16

# 是否允许地址的重用
TCPServer.allow_reuse_address = True
serv = UDPServer(('', 20000), TimeHandler, 
                 # 是否立即绑定并激活 Socket
                 bind_and_activate=False)
# 手动绑定和激活
serv.server_bind()
serv.server_activate()

for _ in range(NWORKERS):
    t = Thread(target=serv.serve_forever)
    t.daemon = True
    t.start()

客户端

In [10]:
from socket import socket, AF_INET, SOCK_DGRAM
s = socket(AF_INET, SOCK_DGRAM)

In [11]:
s.sendto(b'', ('localhost', 20000))

Got connection from ('127.0.0.1', 59844)


0

In [12]:
s.recvfrom(8192)

(b'Thu Nov  1 17:54:02 2018', ('127.0.0.1', 20000))

---

### 通过CIDR地址生成对应的IP地址集

In [13]:
import ipaddress
net = ipaddress.ip_network('123.45.67.64/27')

In [14]:
net

IPv4Network('123.45.67.64/27')

In [15]:
net6 = ipaddress.ip_network('12:3456:78:90ab:cd:ef01:23:30/125')

In [16]:
net6

IPv6Network('12:3456:78:90ab:cd:ef01:23:30/125')

In [17]:
net.num_addresses

32

In [18]:
net[0]

IPv4Address('123.45.67.64')

In [19]:
net[-3]

IPv4Address('123.45.67.93')

In [20]:
a = ipaddress.ip_address('123.45.67.69')

In [21]:
a in net

True

In [22]:
b = ipaddress.ip_address('123.45.67.123')

In [23]:
b in net

False

In [24]:
inet = ipaddress.ip_interface('123.45.67.73/27')

In [25]:
inet

IPv4Interface('123.45.67.73/27')

In [26]:
inet.network

IPv4Network('123.45.67.64/27')

In [27]:
inet.ip

IPv4Address('123.45.67.73')

---

### 实现远程调用 Python 命令

通过 XML-RPC 实现服务端

In [28]:
from xmlrpc.server import SimpleXMLRPCServer


class KeyValueServer:
    _rpc_methods_ = ['get', 'set', 'delete', 'exists', 'keys']

    def __init__(self, address):
        self._data = {}
        self._serv = SimpleXMLRPCServer(address, allow_none=True)
        for name in self._rpc_methods_:
            # 注册所有方法
            self._serv.register_function(getattr(self, name))

    def get(self, name):
        return self._data[name]

    def set(self, name, value):
        self._data[name] = value

    def delete(self, name):
        del self._data[name]

    def exists(self, name):
        return name in self._data

    def keys(self):
        return list(self._data)

    def serve_forever(self):
        self._serv.serve_forever()

In [29]:
from threading import Thread

# 线程数
NWORKERS = 16

serv = KeyValueServer(('', 15000))

for _ in range(NWORKERS):
    t = Thread(target=serv.serve_forever)
    t.daemon = True
    t.start()

客户端需要用代理访问

In [30]:
from xmlrpc.client import ServerProxy
s = ServerProxy('http://localhost:15000', allow_none=True)

In [31]:
s.set('foo', 'bar')

127.0.0.1 - - [01/Nov/2018 17:54:02] "POST /RPC2 HTTP/1.1" 200 -


In [32]:
s.keys()

127.0.0.1 - - [01/Nov/2018 17:54:02] "POST /RPC2 HTTP/1.1" 200 -


['foo']

In [33]:
s.get('foo')

127.0.0.1 - - [01/Nov/2018 17:54:02] "POST /RPC2 HTTP/1.1" 200 -


'bar'

In [34]:
s.delete('foo')

127.0.0.1 - - [01/Nov/2018 17:54:02] "POST /RPC2 HTTP/1.1" 200 -


In [35]:
s.exists('foo')

127.0.0.1 - - [01/Nov/2018 17:54:02] "POST /RPC2 HTTP/1.1" 200 -


False

---
通过传递 pickle 编码实现服务端

In [36]:
import pickle


class RPCHandler:
    """ 基于 pickle 的 RPC 处理器，可以注册远程调用的方法"""

    def __init__(self):
        self._functions = { }

    def register_function(self, func):
        self._functions[func.__name__] = func

    def handle_connection(self, listener):
        try:
            connection = listener.accept()
            while True:
                func_name, args, kwargs = pickle.loads(connection.recv())
                try:
                    r = self._functions[func_name](*args,**kwargs)
                    connection.send(pickle.dumps(r))
                except Exception as e:
                    connection.send(pickle.dumps(e))
        except EOFError:
             pass

要使用这个处理器，你需要将它加入到一个消息服务器中

In [37]:
from multiprocessing.connection import Listener
from threading import Thread

def rpc_server(handler, address, authkey):
    """ RPC 服务器 """
    sock = Listener(address, authkey=authkey)
    for _ in range(16):
        t = Thread(target=handler.handle_connection, args=(sock,))
        t.daemon = True
        t.start()

def add(x, y):
    return x + y

def sub(x, y):
    return x - y

handler = RPCHandler()
handler.register_function(add)
handler.register_function(sub)

rpc_server(handler, ('localhost', 17000), authkey=b'peekaboo')

实现一个专门传送请求的RPC代理类，用于客户端使用

In [38]:
import pickle


class RPCProxy:

    def __init__(self, connection):
        self._connection = connection

    def __getattr__(self, name):
        def do_rpc(*args, **kwargs):
            self._connection.send(pickle.dumps((name, args, kwargs)))
            result = pickle.loads(self._connection.recv())
            if isinstance(result, Exception):
                raise result
            return result
        return do_rpc

客户端

In [39]:
from multiprocessing.connection import Client
c = Client(('localhost', 17000), authkey=b'peekaboo')
proxy = RPCProxy(c)

In [40]:
proxy.add(2, 3)

5

In [41]:
proxy.sub(2, 3)

-1

---

### 在不同的Python解释器之间通信

注意：不要使用 `multiprocessing` 来实现一个对外的公共服务，避免安全问题

In [42]:
from multiprocessing.connection import Listener
import traceback

def echo_client(conn):
    try:
        while True:
            msg = conn.recv()
            conn.send(msg)
    except EOFError:
        print('Connection closed')

def echo_server(address, authkey):
    serv = Listener(address, authkey=authkey)
    while True:
        try:
            client = serv.accept()
            echo_client(client)
        except Exception:
            traceback.print_exc()

服务端启动在线程里，避免阻塞

In [43]:
from threading import Thread

t = Thread(target=echo_server, 
           args=(('', 25000), b'peekaboo'))
t.daemon = True
t.start()

In [44]:
from multiprocessing.connection import Client
c = Client(('localhost', 25000), authkey=b'peekaboo')

In [45]:
c.send('hello')

In [46]:
c.recv()

'hello'

In [47]:
c.send([1, 2, 3, 4, 5])

In [48]:
c.recv()

[1, 2, 3, 4, 5]

In [49]:
c.close()

Connection closed


---

### 简单的客户端认证

可以利用 `hmac` 模块实现一个连接握手，`hmac` 认证的一个常见使用场景是内部消息通信系统和进程间通信。基于 `hmac` 的认证被 `multiprocessing` 模块使用来实现子进程直接的通信。

In [50]:
import hmac
import os

def client_authenticate(connection, secret_key):
    """
    认证客户端
    收到服务端发来的随机字节后，
    通过 hmac 和密钥参数计算哈希，
    然后将哈希发送给服务端
    """
    message = connection.recv(32)
    hash = hmac.new(secret_key, message)
    digest = hash.digest()
    connection.send(digest)

def server_authenticate(connection, secret_key):
    """
    认证服务端
    收到认证请求后，生成一串随机字节发送给客户端，
    然后通过 hmac 和密钥参数计算哈希，等待客户端发来哈希进行对比
    """
    message = os.urandom(32)
    connection.send(message)
    hash = hmac.new(secret_key, message)
    digest = hash.digest()
    response = connection.recv(len(digest))
    # 返回布尔值
    return hmac.compare_digest(digest, response)

将认证函数应用到 socket 中

In [51]:
from socket import socket, AF_INET, SOCK_STREAM

secret_key = b'peekaboo'

def echo_handler(client_sock):
    if not server_authenticate(client_sock, secret_key):
        client_sock.close()
        return
    while True:
        msg = client_sock.recv(8192)
        if not msg:
            break
        client_sock.sendall(msg)

def echo_server(address):
    s = socket(AF_INET, SOCK_STREAM)
    s.bind(address)
    s.listen(5)
    while True:
        c,a = s.accept()
        echo_handler(c)

In [52]:
from threading import Thread

t = Thread(target=echo_server, 
           args=(('', 18000),))
t.daemon = True
t.start()

In [53]:
from socket import socket, AF_INET, SOCK_STREAM

secret_key = b'peekaboo'

s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 18000))
client_authenticate(s, secret_key)

In [54]:
s.send(b'Hello World')

11

In [55]:
s.recv(1024)

b'Hello World'

---

### 在网络服务中加入SSL

实现一个基于 Sockets 的网络服务，客户端和服务器通过 SSL 协议认证并加密传输的数据。  
可以通过定义一个 Mixin 类来添加 SSL

In [56]:
import ssl


class SSLMixin:

    def __init__(self, *args,
                 keyfile=None, certfile=None, ca_certs=None,
                 cert_reqs=ssl.CERT_NONE,
                 **kwargs):
        self._keyfile = keyfile
        self._certfile = certfile
        self._ca_certs = ca_certs
        self._cert_reqs = cert_reqs
        super().__init__(*args, **kwargs)

    def get_request(self):
        client, addr = super().get_request()
        client_ssl = ssl.wrap_socket(
            client,
            keyfile = self._keyfile,
            certfile = self._certfile,
            ca_certs = self._ca_certs,
            cert_reqs = self._cert_reqs,
            server_side = True
        )
        return client_ssl, addr

定义一个基于 SSL 的 XML-RPC 服务器

In [57]:
import ssl
from xmlrpc.server import SimpleXMLRPCServer


class SSLSimpleXMLRPCServer(SSLMixin, SimpleXMLRPCServer):
    pass


class KeyValueServer:
    _rpc_methods_ = ['get', 'set', 'delete', 'exists', 'keys']

    def __init__(self, *args, **kwargs):
        self._data = {}
        self._serv = SSLSimpleXMLRPCServer(
            *args, allow_none=True, **kwargs)
        for name in self._rpc_methods_:
            self._serv.register_function(getattr(self, name))

    def get(self, name):
        return self._data[name]

    def set(self, name, value):
        self._data[name] = value

    def delete(self, name):
        del self._data[name]

    def exists(self, name):
        return name in self._data

    def keys(self):
        return list(self._data)

    def serve_forever(self):
        self._serv.serve_forever()

这里为了测试，需要先创建自签名证书：  
`openssl req -new -x509 -days 365 -nodes -out server_cert.pem -keyout server_key.pem`

然后再复制并重命名一份 client 证书，一起保存在当前目录下

In [58]:
from threading import Thread

KEYFILE='server_key.pem'
CERTFILE='server_cert.pem'
CA_CERTS='client_cert.pem'

kvserv = KeyValueServer(('', 16000),
                        keyfile=KEYFILE,
                        certfile=CERTFILE,
                        ca_certs=CA_CERTS,
                        cert_reqs=ssl.CERT_REQUIRED)

t = Thread(target=kvserv.serve_forever)
t.daemon = True
t.start()

In [59]:
from xmlrpc.client import SafeTransport, ServerProxy
import ssl


class VerifyCertSafeTransport(SafeTransport):

    def __init__(self, cafile, certfile=None, keyfile=None):
        SafeTransport.__init__(self)
        self._ssl_context = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
        self._ssl_context.load_verify_locations(cafile)
        if certfile:
            self._ssl_context.load_cert_chain(certfile, keyfile)
        self._ssl_context.verify_mode = ssl.CERT_REQUIRED
        self.context = self._ssl_context

In [60]:
try:
    s = ServerProxy('https://localhost:16000',
                    transport=VerifyCertSafeTransport('server_cert.pem',
                                                      'client_cert.pem',
                                                      'client_key.pem'),
                    allow_none=True)
    print('Set:', s.set('foo','bar'))
    print('Get:', s.get('foo'))
except FileNotFoundError:
    print('Error: pem file not found')

Error: pem file not found


---

### 进程间传递 Socket 文件描述符

当有多个 Python 解释器进程在同时运行，需求将某个打开的文件描述符从一个解释器传递给另外一个。这个实现通常使用 `multiprocessing` 模块来创建进程之间的管道连接，然后使用 `multiprocessing.reduction` 中的 `send_handle` 和 `recv_handle` 函数在不同的处理器直接传递文件描述符

In [61]:
import multiprocessing
from multiprocessing.reduction import recv_handle, send_handle
import socket

def worker(in_p, out_p):
    """
    工作进程
    通过 recv_handle 接收一个文件描述符，
    接收到后创建 Socket 并向客户端回应数据
    """
    out_p.close()
    while True:
        fd = recv_handle(in_p)
        print('CHILD: GOT FD', fd)
        with socket.socket(
            socket.AF_INET, socket.SOCK_STREAM, fileno=fd) as s:
            while True:
                msg = s.recv(1024)
                if not msg:
                    break
                print('CHILD: RECV {!r}'.format(msg))
                s.send(msg)

def server(address, in_p, out_p, worker_pid):
    """
    服务进程
    创建 Socket 并等待工作进程的连接请求，接收到请求后，
    将产生的 Socket 文件描述符通过 send_handle 传递给工作进程
    """
    in_p.close()
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
    s.bind(address)
    s.listen(1)
    while True:
        client, addr = s.accept()
        print('SERVER: Got connection from', addr)
        send_handle(out_p, client.fileno(), worker_pid)
        client.close()

In [62]:
c1, c2 = multiprocessing.Pipe()
worker_p = multiprocessing.Process(target=worker, args=(c1,c2))
worker_p.start()

server_p = multiprocessing.Process(target=server,
              args=(('', 19000), c1, c2, worker_p.pid))
server_p.start()

c1.close()
c2.close()

---

### 理解事件驱动的 I/O

事件驱动 I/O 本质上来讲就是将基本 I/O 操作（比如读和写）转化为程序需要处理的事件  
举例实现一系列基本事件处理器

In [63]:
class EventHandler:

    def fileno(self):
        raise NotImplemented('must implement')

    def wants_to_receive(self):
        return False

    def handle_receive(self):
        pass

    def wants_to_send(self):
        return False

    def handle_send(self):
        pass

事件循环函数，其关键部分是 `select` 调用，它会不断轮询文件描述符从而激活它

In [64]:
import select

def event_loop(handlers):
    while True:
        # 先询问所有的处理器来决定哪一个想接受或发送
        wants_recv = [h for h in handlers if h.wants_to_receive()]
        wants_send = [h for h in handlers if h.wants_to_send()]
        # 通过 select 返回准备接受或发送的对象组成的列表
        can_recv, can_send, _ = select.select(wants_recv, wants_send, [])
        # 触发对应方法
        for h in can_recv:
            h.handle_receive()
        for h in can_send:
            h.handle_send()

基于UDP网络服务的处理器

In [65]:
import socket
import time


class UDPServer(EventHandler):

    def __init__(self, address):
        self.sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.sock.bind(address)

    def fileno(self):
        return self.sock.fileno()

    def wants_to_receive(self):
        return True


class UDPTimeServer(UDPServer):

    def handle_receive(self):
        msg, addr = self.sock.recvfrom(1)
        self.sock.sendto(time.ctime().encode('ascii'), addr)


class UDPEchoServer(UDPServer):

    def handle_receive(self):
        msg, addr = self.sock.recvfrom(8192)
        self.sock.sendto(msg, addr)

In [66]:
from threading import Thread

handlers = [UDPTimeServer(('', 21000)), UDPEchoServer(('', 22000))]

t = Thread(target=event_loop, args=(handlers,))
t.daemon = True
t.start()

In [67]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_DGRAM)

In [68]:
s.sendto(b'', ('localhost', 21000))

0

In [69]:
s.recvfrom(128)

(b'Thu Nov  1 17:54:02 2018', ('127.0.0.1', 21000))

In [70]:
s.sendto(b'Hello', ('localhost', 22000))

5

In [71]:
s.recvfrom(128)

(b'Hello', ('127.0.0.1', 22000))

---
TCP服务器复杂一些，因为每一个客户端都要初始化一个新的处理器对象

In [72]:
import socket


class TCPServer(EventHandler):

    def __init__(self, address, client_handler, handler_list):
        self.sock = socket.socket(
            socket.AF_INET, socket.SOCK_STREAM)
        self.sock.setsockopt(
            socket.SOL_SOCKET, socket.SO_REUSEADDR, True)
        self.sock.bind(address)
        self.sock.listen(1)
        self.client_handler = client_handler
        self.handler_list = handler_list

    def fileno(self):
        return self.sock.fileno()

    def wants_to_receive(self):
        return True

    def handle_receive(self):
        client, addr = self.sock.accept()
        # 对于每一个连接，新的处理器被添加到列表中
        self.handler_list.append(
            self.client_handler(client, self.handler_list))


class TCPClient(EventHandler):

    def __init__(self, sock, handler_list):
        self.sock = sock
        self.handler_list = handler_list
        self.outgoing = bytearray()

    def fileno(self):
        return self.sock.fileno()

    def close(self):
        self.sock.close()
        # 当连接被关闭后，客户端需要将自身从列表中删除
        self.handler_list.remove(self)

    def wants_to_send(self):
        return True if self.outgoing else False

    def handle_send(self):
        nsent = self.sock.send(self.outgoing)
        self.outgoing = self.outgoing[nsent:]


class TCPEchoClient(TCPClient):

    def wants_to_receive(self):
        return True

    def handle_receive(self):
        data = self.sock.recv(8192)
        if not data:
            self.close()
        else:
            self.outgoing.extend(data)

In [73]:
from threading import Thread

handlers = []
handlers.append(TCPServer(('', 23000), TCPEchoClient, handlers))

t = Thread(target=event_loop, args=(handlers,))
t.daemon = True
t.start()

In [74]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.connect(('localhost', 23000))

In [75]:
s.send(b'Hello')

5

In [76]:
s.recv(8192)

b'Hello'

---
当面对事件阻塞或耗时计算的问题，需要通过将事件发送给其他单独的线程或进程来处理

In [77]:
from concurrent.futures import ThreadPoolExecutor
import socket
import os


class ThreadPoolHandler(EventHandler):

    def __init__(self, nworkers):
        """ 创建一对 Socket 作为信号量机制，来协调计算结果和事件循环"""
        if os.name == 'posix':
            self.signal_done_sock, self.done_sock = socket.socketpair()
        else:
            server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            server.bind(('127.0.0.1', 0))
            server.listen(1)
            self.signal_done_sock = socket.socket(socket.AF_INET,
                                                  socket.SOCK_STREAM)
            self.signal_done_sock.connect(server.getsockname())
            self.done_sock, _ = server.accept()
            server.close()

        self.pending = []
        self.pool = ThreadPoolExecutor(nworkers)

    def fileno(self):
        return self.done_sock.fileno()

    def _complete(self, callback, r):
        """ 将挂起的回调函数和结果放入队列中，然后发出信号 """
        self.pending.append((callback, r.result()))
        self.signal_done_sock.send(b'x')

    def run(self, func, args=(), kwargs={},*,callback):
        """ 将工作提交给线程池，完成后回调 _complete 方法 """
        r = self.pool.submit(func, *args, **kwargs)
        r.add_done_callback(lambda r: self._complete(callback, r))

    def wants_to_receive(self):
        return True

    def handle_receive(self):
        for callback, result in self.pending:
            callback(result)
            self.done_sock.recv(1)
        self.pending = []

通过计算斐波那契数列，来测试多线程的事件循环

In [78]:
def fib(n):
    if n < 2:
        return 1
    else:
        return fib(n - 1) + fib(n - 2)


class UDPFibServer(UDPServer):

    def handle_receive(self):
        msg, addr = self.sock.recvfrom(128)
        n = int(msg)
        pool.run(fib, (n,), callback=lambda r: self.respond(r, addr))

    def respond(self, result, addr):
        self.sock.sendto(str(result).encode('ascii'), addr)

In [79]:
from threading import Thread

pool = ThreadPoolHandler(16)
handlers = [pool, UDPFibServer(('', 24000))]

t = Thread(target=event_loop, args=(handlers,))
t.daemon = True
t.start()

In [80]:
from socket import *

sock = socket(AF_INET, SOCK_DGRAM)

for x in range(35):
    sock.sendto(str(x).encode('ascii'), ('localhost', 24000))
    resp = sock.recvfrom(8192)
    print(resp[0], end=' ')

b'1' b'1' b'2' b'3' b'5' b'8' b'13' b'21' b'34' b'55' b'89' b'144' b'233' b'377' b'610' b'987' b'1597' b'2584' b'4181' b'6765' b'10946' b'17711' b'28657' b'46368' b'75025' b'121393' b'196418' b'317811' b'514229' b'832040' b'1346269' b'2178309' b'3524578' b'5702887' b'9227465' 

---

### 发送与接收大型数组

In [81]:
def send_from(arr, dest):
    # 将数组转换为无符号字节的内存视图
    view = memoryview(arr).cast('B')
    while len(view):
        nsent = dest.send(view)
        view = view[nsent:]

def recv_into(arr, source):
    view = memoryview(arr).cast('B')
    while len(view):
        nrecv = source.recv_into(view)
        view = view[nrecv:]

服务端

In [82]:
from socket import socket, AF_INET, SOCK_STREAM
s = socket(AF_INET, SOCK_STREAM)
s.bind(('', 26000))
s.listen(1)

客户端

In [83]:
from socket import socket, AF_INET, SOCK_STREAM
c = socket(AF_INET, SOCK_STREAM)
c.connect(('localhost', 26000))

服务端发送数组

In [84]:
from threading import Thread
import numpy

s_a = numpy.arange(0.0, 5000000.0)

t = Thread(target=send_from, args=(s_a, c))
t.daemon = True
t.start()

客户端需要预知数组的长度，然后接收数组

In [85]:
from threading import Thread
import numpy

c_a = numpy.zeros(shape=50000000, dtype=float)

t = Thread(target=recv_into, args=(c_a, c))
t.daemon = True
t.start()

In [86]:
c_a[0:10]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])